# Data Preparation

In [1]:
from src.colors import bcolors
from config import Config

c = bcolors()
config = Config()

CKPT_PATH = 'checkpoints/unet/'

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from src.datasets.EuroSatMS10 import EuroSatMS10

df = pd.read_csv(config.TRAIN_FILE)

x_chan = [10, 8, 7]
y_chan = 9

dataset = EuroSatMS10(
    df,
    config.TRAIN_MS_DIR,
    x_chan, y_chan,
    scaler=True
)

ds_train, ds_val = train_test_split(dataset, test_size=0.2, stratify=df['label'])

print(f"""\n{c.OKCYAN}Train dataset: {len(ds_train)} samples{c.ENDC}""")
print(f"""{c.OKCYAN}Validation dataset: {len(ds_val)} samples{c.ENDC}""")


Preloading images...
Number of images: 27000
Number of jobs:   -4 

Time taken:      0 min 40.66543507575989 sec 

MinMax scaling the images...

Train dataset: 21600 samples
Validation dataset: 5400 samples


In [3]:
from src.training.data import EuroSatDataModule

BATCH_SIZE = 64
N_WORKERS = 10

print(f"""{c.OKGREEN}Initializing the data module...{c.ENDC}""")
data_module = EuroSatDataModule(ds_train, ds_val, BATCH_SIZE, N_WORKERS)

Initializing the data module...


# Training U-Net model

In [4]:
from datetime import datetime
from pytorch_lightning.callbacks import ModelCheckpoint
from src.training.unet import UNetLightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

logger = WandbLogger(
    project="eurosat_unet",
    name="unet_v1",
    log_model=False,
)

checkpoint_callback = ModelCheckpoint(
    dirpath=CKPT_PATH + datetime.now().strftime("%H-%M"),
    save_top_k=1, 
    monitor="val_loss",
    verbose=False
)

trainer = Trainer(
    max_epochs=20,
    accelerator="gpu", 
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback],
)

lightning_model = UNetLightningModule(
    in_channels=len(x_chan),
    learning_rate=0.0001,
    gamma=0.8,
    weight_decay=0.00001,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
for param in lightning_model.model.encoder.parameters():
    param.requires_grad = False

trainer.fit(lightning_model, datamodule=data_module)

wandb: Currently logged in as: the-virus. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type | Params
-----------------------------------
0 | model     | Unet | 24.4 M
1 | criterion | SSIM | 0     
-----------------------------------
3.2 M     Trainable params
21.3 M    Non-trainable params
24.4 M    Total params
97.745    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


In [14]:
lightning_model = UNetLightningModule.load_from_checkpoint(
    checkpoint_callback.best_model_path,
    in_channels=len(x_chan),
    learning_rate=0.0001,
    gamma=0.5,
    weight_decay=0.00001,
)

for param in lightning_model.model.parameters():
    param.requires_grad = True
    
trainer = Trainer(
    max_epochs=5,
    accelerator="gpu", 
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback],
)

trainer.fit(lightning_model, datamodule=data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/tobias/Schreibtisch/Uni/HSG/ML/EuroSatClassification/.venv/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /home/tobias/Schreibtisch/Uni/HSG/ML/EuroSatClassification/checkpoints/unet/17-02 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type | Params
-----------------------------------
0 | model     | Unet | 24.4 M
1 | criterion | SSIM | 0     
-----------------------------------
24.4 M    Trainable params
0         Non-trainable params
24.4 M    Total params
97.745    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/home/tobias/Schreibtisch/Uni/HSG/ML/EuroSatClassification/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


# Evaluate Model Predictions

In [13]:
import torch
from fastai.data.load import DataLoader
from matplotlib import pyplot as plt
from src.training.unet import UNetLightningModule
import numpy as np

dataloader = DataLoader(ds_val, batch_size=32, shuffle=False)

model = UNetLightningModule.load_from_checkpoint(
    checkpoint_callback.best_model_path,
    in_channels=len(x_chan),
    learning_rate=0.000005,
    gamma=0.9,
    weight_decay=0.00001,
)

model.model.eval()

with torch.no_grad():
    for batch in dataloader:
        x, y = batch
        x = x.to(model.device)
        out = model.model(x)
        print(out.shape)
        print(y.shape)
        
        
        y_np = y.cpu()
        out_np = out.cpu()
        
        fig, axes = plt.subplots(len(y_np), 2, figsize=(10, 100))
        for i in range(len(y_np)):
            target_img = torch.stack([y_np[i], y_np[i], y_np[i]], dim=0).permute(1, 2, 0)
            rgb_min = target_img.min()
            rgb_max = target_img.max()
    
            target_img = (target_img - rgb_min) / (rgb_max - rgb_min)

            axes[i, 0].imshow(target_img, cmap='gray')
            axes[i, 0].set_title(f'Target')
            axes[i, 0].axis('off')
            
            outp_img = torch.stack([out_np[i][0], out_np[i][0], out_np[i][0]], dim=0).permute(1, 2, 0)
            rgb_min = outp_img.min()
            rgb_max = outp_img.max()
    
            outp_img = (outp_img - rgb_min) / (rgb_max - rgb_min)
            
            outp_img = np.round(outp_img * 8) / 8
            
            axes[i, 1].imshow(outp_img, cmap='gray')
            axes[i, 1].set_title(f'Output')
            axes[i, 1].axis('off')
    
        plt.show()
        break

torch.Size([32, 1, 64, 64])
torch.Size([32, 64, 64])
